In [1]:
import nltk
import pandas as pd
import string

In [2]:
import multidict as multidict

import numpy as np

import os
import re
from PIL import Image
from os import path
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [3]:
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Reen\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
df = pd.read_csv('/results/results.csv')

In [21]:
new_df = pd.DataFrame(columns=['ID','adj_word_cloud_dd','adj_word_cloud_ndtv','adj_word_cloud_tn'])

In [13]:
#df.head()

In [14]:
def lemmatize_entity(entity):
    result = (entity.lower()).translate(str.maketrans("","", string.punctuation))
    word_list = nltk.word_tokenize(result)
    filtered_output = [w for w in word_list if not w in stop_words]
    lemmatized_key = [lemmatizer.lemmatize(w) for w in filtered_output]
    return lemmatized_key

In [15]:
def get_adjectives(src_tuple_list):
    adj_list = []
    for item in range(len(src_tuple_list)):
        if(src_tuple_list[item][1]=="JJ"):
            adj_list.append(src_tuple_list[item][0])
    return adj_list

In [16]:
def getFrequencyDictForText(sentence):
    fullTermsDict = multidict.MultiDict()
    tmpDict = {}

    # making dict for counting frequencies
    for text in sentence.split(" "):
        if re.match("a|the|an|the|to|in|for|of|or|by|with|is|on|that|be", text):
            continue
        val = tmpDict.get(text, 0)
        tmpDict[text.lower()] = val + 1
    for key in tmpDict:
        fullTermsDict.add(key, tmpDict[key])
    return fullTermsDict

In [22]:
for index,row in df.iterrows():
    #print('hello')
    news_item_id = row['ID']
    dd_article = row['dd_text']
    ndtv_article = row['ndtv_text']
    times_article = row['tn_text']
    
    lemma_dd = lemmatize_entity(dd_article)
    lemma_ndtv = lemmatize_entity(ndtv_article)
    lemma_times = lemmatize_entity(times_article)
    
    
    ddadj = nltk.pos_tag(lemma_dd)
    ndtvadj =  nltk.pos_tag(lemma_ndtv)
    tnadj = nltk.pos_tag(lemma_times)
    
    dd_adj = get_adjectives(ddadj)
    ndtv_adj = get_adjectives(ndtvadj)
    tn_adj = get_adjectives(tnadj)
    
    dd_adj = (' '.join(dd_adj))
    ndtv_adj = (' '.join(ndtv_adj))
    tn_adj = (' '.join(tn_adj))
    
    dd_dict = dict(getFrequencyDictForText(dd_adj))
    ndtv_dict = dict(getFrequencyDictForText(ndtv_adj))
    times_dict = dict(getFrequencyDictForText(tn_adj))
    
    new_df = new_df.append({'ID': news_item_id, 'adj_word_cloud_dd':dd_dict, 'adj_word_cloud_ndtv':ndtv_dict, 'adj_word_cloud_tn':times_dict},ignore_index=True)

In [24]:
new_df.to_csv('./adjectives_word_cloud_data.csv')